In [0]:
!nvidia-smi
# Download image and class labels
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/img.jpg
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/labels_map.txt

In [0]:
# Get EfficientNet PyTorch
!pip install efficientnet_pytorch

In [0]:
import json
from PIL import Image
import numpy as np

import torch
import torchvision
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
#from torchvision import transforms

from efficientnet_pytorch import EfficientNet

In [0]:
seed = 0
gpu = 0

np.random.seed(seed)
torch.cuda.set_device(gpu)
cudnn.benchmark = True
torch.manual_seed(seed)
cudnn.enabled = True
torch.cuda.manual_seed(seed)

model_name = 'efficientnet-b0'
image_size = EfficientNet.get_image_size(model_name) # 224

In [0]:
img = Image.open('img.jpg')
#img

In [0]:
# Preprocess image
tfms = transforms.Compose([transforms.Resize(image_size), transforms.CenterCrop(image_size), 
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(img).unsqueeze(0)
img = Variable(img)
img = img.cuda()

In [0]:
# Load class names
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

In [0]:
# Classify with EfficientNet
model = EfficientNet.from_pretrained(model_name)
model = model.cuda()
#model.cuda()
model.eval()
with torch.no_grad():
    logits = model(img)
preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()

print('-----')
for idx in preds:
    label = labels_map[idx]
    prob = torch.softmax(logits, dim=1)[0, idx].item()
    print('{:<75} ({:.2f}%)'.format(label, prob*100))

# How to profiling layer-by-layer in Pytroch?
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    model(img)
print(prof) 

In [0]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof.tgz
!tar zxvf imagewoof.tgz


In [0]:
!ls ./imagewoof/val/

In [0]:
data_sub = './imagewoof/val/'
batch_size = 1
test_transform = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BICUBIC),
            transforms.ToTensor(),
        ])        
test_data = torchvision.datasets.ImageFolder(root=data_sub, transform=test_transform)

num_test = len(test_data)
print (num_test)
test_queue = torch.utils.data.DataLoader(
            test_data,
            batch_size=batch_size,
            shuffle=False,
            pin_memory=1)

In [0]:
confusion_matrix = torch.zeros(10, 895)
with torch.no_grad():

   for step, (input, target) in enumerate(test_queue):
       input = Variable(input).cuda()
       target = Variable(target).cuda(async=True)
       #input = Variable(input)
       #target = Variable(target)

       outputs = model(input)
       _, preds = torch.max(outputs, 1)
       #print(preds)
       for t, p in zip(target.view(-1), preds.view(-1)):
           #print(t, p)
           confusion_matrix[t.long(), p.long()] += 1
#print(confusion_matrix)

In [0]:
#print(target.view(-1))
#print(preds.view(-1))
#print(confusion_matrix[0,155]>=1)
for t in range (10):
 for p in range (895): 
  if confusion_matrix[t,p]>=1:
   print(t, p, confusion_matrix[t,p])

In [0]:
#print(confusion_matrix.diag()/confusion_matrix.sum(1))